---
# Transformer XL
---

---
# Import Libraries

In [1]:
import os
import sys

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [3]:
sys.path.append("../../deep-learning-dna")

In [4]:
import wandb

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import math
import string

import Set_Transformer 
from common.models import dnabert
from common import dna
from lmdbm import Lmdb
from common.data import DnaSequenceGenerator, DnaLabelType, DnaSampleGenerator, find_dbs
import wandb

import tf_utils as tfu

In [6]:
strategy = tfu.devices.select_gpu(0, use_dynamic_memory=True)

---
# Load Data

In [7]:
#Import pretrained model
api = wandb.Api()
model_path = api.artifact("sirdavidludwig/dnabert-pretrain/dnabert-pretrain-8dim:latest").download()
pretrained_model = dnabert.DnaBertModel.load(model_path)
pretrained_model.load_weights(model_path + "/model.h5")
pretrained_model

In [8]:
#Load datafiles
dataset_path = api.artifact("sirdavidludwig/nachusa-dna/dnasamples-complete:latest").download('/data/dna_samples:v1')
samples = find_dbs(dataset_path + '/train')
samples[13]

wandb: Downloading large artifact dnasamples-complete:latest, 4079.09MB. 420 files... Done. 0:0:0.1


'/data/dna_samples:v1/train/WS-CCW-Jul2015_S82_L001_R1_001.db'

---
# Create Dataset

In [9]:
split_ratios = [0.8, 0.2]
set_len = 1000
sequence_len = 150
kmer = 3
batch_size = [20,5]
batches_per_epoch = 20
augument = True
labels = DnaLabelType.SampleIds
seed = 0
rng = np.random.default_rng(seed)
random_samples = samples.copy()

In [10]:
rng.shuffle(random_samples)

In [11]:
trimmed_samples, (train_dataset, val_dataset) = DnaSampleGenerator.split(samples=random_samples[0:50], split_ratios=split_ratios, subsample_length=set_len, sequence_length=sequence_len, kmer=kmer,batch_size=batch_size,batches_per_epoch=batches_per_epoch,augment=augument,labels=labels, rng=rng)

Sample '/data/dna_samples:v1/train/Wes7-PCRblank1_S8_L001_R1_001.db' does not contain enough sequences. This sample will be ignored.
Sample '/data/dna_samples:v1/train/Wes7-PCRblank1_S8_L001_R1_001.db' does not contain enough sequences. This sample will be ignored.


In [12]:
random_samples[0:50]

['/data/dna_samples:v1/train/WS-CCE-Apr2016_S6_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes52-10-TC_S53_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-WH-Jul2016_S46_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes41-10-HN_S42_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wesley026-Ag-072820_S165_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-MU-Apr2016_S84_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes5-5-CCE_S6_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-MR-Apr2016_S13_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-MU-Sep2015_S43_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wesley012-HN-051120_S151_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-HPN-Sep2015_S91_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-TCR-Sep2015_S52_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes26-8-AG_S27_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-SB-Jul2016_S22_L001_R1_001.db',
 '/data/dna_samples:v1/train/Wes25-8-MU_S26_L001_R1_001.db',
 '/data/dna_samples:v1/train/WS-SB-Oct

--- 
# Batch Parameters

In [13]:
block_size = 200
seq_len = set_len
maxlen = set_len
vocab_size = 5
num_chars_data = set_len*sequence_len
max_files = len(train_dataset.samples)
max_files

49

In [14]:
if block_size-2 > seq_len:
    raise ValueError("Block size should not be bigger than sequence length")

In [15]:
print(maxlen)
print(vocab_size)
print(num_chars_data)

1000
5
150000


---
# Create Embeddings

In [16]:
#Create 8 dimensional embeddings
pretrained_encoder = dnabert.DnaBertEncoderModel(pretrained_model.base)
pretrained_encoder.trainable = False

In [17]:
class Create_Embeddings(keras.layers.Layer):
    def __init__(self, encoder):
        super(Create_Embeddings, self).__init__()
        self.encoder = encoder
        
    def subbatch_predict(self, model, batch, subbatch_size, concat=lambda old, new: tf.concat((old, new), axis=0)):
        def predict(i, result=None):
            n = i + subbatch_size
            pred = tf.stop_gradient(model(batch[i:n]))
            if result is None:
                return [n, pred]
            return [n, concat(result, pred)]
        i, result = predict(0)
        batch_size = tf.shape(batch)[0]
        i, result = tf.while_loop(
            cond=lambda i, _: i < batch_size,
            body=predict,
            loop_vars=[i, result],
            parallel_iterations=1)

        return result
    
    def modify_data_for_input(self, data):
        batch_size = tf.shape(data)[0]
        subsample_size = tf.shape(data)[1]
        flat_data = tf.reshape(data, (batch_size*subsample_size, -1))
        encoded = self.subbatch_predict(self.encoder, flat_data, 128)
        return tf.reshape(encoded, (batch_size, subsample_size, -1))
    
    def call(self, data):
        return  self.modify_data_for_input(data)

---
# Setup

In [18]:
def clone_initializer(initializer):
    if isinstance(initializer, tf.keras.initializers.Initializer):
        return initializer.__class__.from_config(initializer.get_config())
    return initializer

---
# Cache Memory

In [19]:
def _cache_memory(current_state, previous_state, memory_length, reuse_length=0):
    if memory_length is None or memory_length == 0:
        return None
    else:
        if reuse_length > 0:
            current_state = current_state[:, :reuse_length, :]

        if previous_state is None:
            new_mem = current_state[:, -memory_length:, :]
        else:
            new_mem = tf.concat(
                    [previous_state, current_state], 1)[:, -memory_length:, :]

    return tf.stop_gradient(new_mem)

---
# XL Block

In [20]:
class TransformerXLBlock(tf.keras.layers.Layer):
    def __init__(self,
                 stack,
                 num_induce,
                 vocab_size,
                 hidden_size,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 norm_epsilon=1e-12,
                 inner_activation="relu",
                 kernel_initializer="variance_scaling",
                 inner_dropout=0.0,
                 **kwargs):

        super(TransformerXLBlock, self).__init__(**kwargs)
        self._stack = stack
        self._num_induce = num_induce
        self._vocab_size = vocab_size
        self._num_heads = num_attention_heads
        self._head_size = head_size
        self._hidden_size = hidden_size
        self._inner_size = inner_size
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._inner_activation = inner_activation
        self._norm_epsilon = norm_epsilon
        self._kernel_initializer = kernel_initializer
        self._inner_dropout = inner_dropout
        self._attention_layer_type = Set_Attention
        
    def build(self, input_shape):
        input_tensor = input_shape[0] if len(input_shape) == 2 else input_shape
        input_tensor_shape = tf.TensorShape(input_tensor)
        if len(input_tensor_shape.as_list()) != 3:
            raise ValueError("TransformerLayer expects a three-dimensional input of "
                                             "shape [batch, sequence, width].")
        batch_size, sequence_length, hidden_size = input_tensor_shape

        if hidden_size % self._num_heads != 0:
            raise ValueError(
                    "The input size (%d) is not a multiple of the number of attention "
                    "heads (%d)" % (hidden_size, self._num_heads))
            
            
        self._attention_layer= self._attention_layer_type(num_induce=self._num_induce, embed_dim=self._head_size,num_heads=self._num_heads, stack=self._stack,
                                                   use_layernorm=True,pre_layernorm=True,use_keras_mha=True)
        
        super(TransformerXLBlock, self).build(input_shape)

   
    def call(self,
             content_stream,
             content_attention_bias,
             positional_attention_bias,
             relative_position_encoding=None,
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        attention_output = {"content_attention": self._attention_layer(content_stream, state)}        

        return attention_output

---
# Transformer XL

In [21]:
class TransformerXL(tf.keras.layers.Layer):
    def __init__(self,
                 stack, 
                 num_induce, 
                 vocab_size,
                 num_layers,
                 hidden_size,
                 maxlen,
                 embed_dim,
                 num_attention_heads,
                 head_size,
                 inner_size,
                 dropout_rate,
                 attention_dropout_rate,
                 initializer,
                 tie_attention_biases=True,
                 memory_length=None,
                 reuse_length=None,
                 inner_activation="relu",
                 **kwargs):
        super(TransformerXL, self).__init__(**kwargs)

        self._stack = stack
        self._num_induce = num_induce
        self._vocab_size = vocab_size
        self._initializer = initializer
        self._num_layers = num_layers
        self._hidden_size = hidden_size
        self._num_attention_heads = num_attention_heads
        self._head_size = head_size
        self._inner_size = inner_size
        self._inner_activation = inner_activation
        self._dropout_rate = dropout_rate
        self._attention_dropout_rate = attention_dropout_rate
        self._tie_attention_biases = tie_attention_biases
        self._memory_length = memory_length
        self._reuse_length = reuse_length

        self.transformer_xl_layers = []
        
        for i in range(self._num_layers):
            self.transformer_xl_layers.append(
                    TransformerXLBlock(
                            stack=self._stack,
                            num_induce=num_induce, 
                            vocab_size=self._vocab_size,
                            hidden_size=self._head_size * self._num_attention_heads,
                            num_attention_heads=self._num_attention_heads,
                            head_size=self._head_size,
                            inner_size=self._inner_size,
                            dropout_rate=self._dropout_rate,
                            attention_dropout_rate=self._attention_dropout_rate,
                            norm_epsilon=1e-12,
                            inner_activation=self._inner_activation,
                            kernel_initializer="variance_scaling",
                            name="layer_%d" % i))

        self.output_dropout = tf.keras.layers.Dropout(rate=self._dropout_rate)

    def call(self,
             content_stream,
             relative_position_encoding,
             state=None,
             content_attention_mask=None,
             query_attention_mask=None,
             target_mapping=None):
        
        new_mems = []

        if state is None:
            state = [None] * self._num_layers
            
        for i in range(self._num_layers):
            new_mems.append(_cache_memory(content_stream, state[i], self._memory_length, self._reuse_length))
            
            transformer_xl_layer = self.transformer_xl_layers[i]
            
            transformer_xl_output = transformer_xl_layer(
                    content_stream=content_stream,
                    content_attention_bias=None,
                    positional_attention_bias=None,
                    relative_position_encoding=None,
                    state=state[i],
                    content_attention_mask=content_attention_mask,
                    query_attention_mask=query_attention_mask,
                    target_mapping=target_mapping)
            
            content_stream = transformer_xl_output["content_attention"]

        output_stream = content_stream
        return output_stream, new_mems

In [22]:
class Set_Attention(keras.Model):
    def __init__(self, num_induce, embed_dim, num_heads, stack, use_layernorm, pre_layernorm, use_keras_mha):
        super(Set_Attention, self).__init__()
        
        self.set_attention = (Set_Transformer.InducedSetAttentionBlock(embed_dim=embed_dim,num_heads=num_heads, num_induce=num_induce, use_layernorm=use_layernorm,pre_layernorm=pre_layernorm,use_keras_mha=use_keras_mha))
      
    def call(self, data, mems):
                
            attention = self.set_attention([data, mems])
                
            return attention

---
# Xl Model Class

In [23]:
class XlModel(keras.Model):
    def __init__(self, stack, num_induce, max_files, encoder, block_size, seq_len_padded, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer):
        super(XlModel, self).__init__()
        
        self.block_size = block_size
        self.seq_len_padded = seq_len_padded
        self.embed_dim = embed_dim
        self.num_heads = num_attention_heads
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.maxlen = maxlen
        self.memory_length = memory_length
        self.max_files = max_files
        self.encoder = encoder
        self.isabs = []
        
        self.embedding_layer = Create_Embeddings(encoder)

        self.linear_layer = keras.layers.Dense(embed_dim)
        
        self.transformer_xl = TransformerXL(
                stack=stack, 
                num_induce=num_induce, 
                vocab_size=vocab_size,
                num_layers=num_layers,
                hidden_size=hidden_size,
                num_attention_heads=num_attention_heads,
                maxlen=maxlen,
                embed_dim=embed_dim,
                memory_length=memory_length,
                reuse_length=reuse_length,
                head_size=head_size,
                inner_size=inner_size,
                dropout_rate=dropout_rate,
                attention_dropout_rate=attention_dropout_rate,
                initializer=initializer, 
            )
        

        self.pooling_layer = Set_Transformer.PoolingByMultiHeadAttention(num_seeds=1,embed_dim=embed_dim,num_heads=1,use_layernorm=True,pre_layernorm=True, use_keras_mha=True,is_final_block=True)
    
        self.reshape_layer = keras.layers.Reshape((embed_dim,))
   
        self.output_layer = keras.layers.Dense(self.max_files, activation=keras.activations.softmax)
        
    
    def call(self, x, training=None):        
 
        mems = tf.zeros((self.num_layers, tf.shape(x)[0], self.memory_length, self.embed_dim))
        
        embeddings = self.embedding_layer(x)
            
        linear_transform = self.linear_layer(embeddings)    
            
        for i in range(0, self.seq_len_padded, self.block_size):
            block = embeddings[:,i:i+self.block_size]
            
            output, mems = self.transformer_xl(content_stream=block, relative_position_encoding=None, state=mems)
                
        pooling = self.pooling_layer(output)

        reshape = self.reshape_layer(pooling)

        output = self.output_layer(reshape)          
        
        return output

---
# Xl Parameters

In [24]:
#Xl Parameters
stack = 1
num_induce = 30
embed_dim = 8
num_layers = 8
hidden_size = 32
num_attention_heads = 8
memory_length = 200
reuse_length = 0
head_size = 8
inner_size = 32
dropout_rate = 0.01
attention_dropout_rate = 0.01
initializer = keras.initializers.RandomNormal(stddev=0.1) 

encoder = pretrained_encoder
epochs = 10000

In [27]:
Parameters = dict(
    stack = 1,
    num_induce = 30,
    embed_dim = 8,
    num_layers = 8,
    hidden_size = 32,
    num_attention_heads = 8,
    memory_length = 200,
    reuse_length = 0,
    head_size = 8,
    inner_size = 32,
    dropout_rate = 0.01,
    attention_dropout_rate = 0.01,
    block_size = block_size,
    set_len=set_len)

In [28]:
run = wandb.init(project="Str_XL_Induced_Mha", config=Parameters, id='15halbt2', resume='allow')

wandb: Currently logged in as: kendragivens. Use `wandb login --relogin` to force relogin


In [31]:
initial_epoch = run.summary.epoch

---
# Create Models

In [32]:
model = XlModel(stack, num_induce, max_files, encoder, block_size, seq_len, embed_dim, vocab_size, num_layers, hidden_size, num_attention_heads, maxlen, memory_length, reuse_length, head_size, inner_size, dropout_rate, attention_dropout_rate, initializer)
model.compile(loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False), optimizer = keras.optimizers.Nadam(1e-4), metrics = keras.metrics.SparseCategoricalAccuracy())

In [35]:
model.load_weights("./Saved_Models/Str_Xl_Induced.h5")

In [ ]:
history = model.fit(initial_epoch=initial_epoch, x=train_dataset, validation_data=val_dataset, epochs=epochs, verbose=1, callbacks=[wandb.keras.WandbCallback(save_weights_only=True)])

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.
wandb: WARNING Unable to compute FLOPs for this model.


Epoch 644/10000
20/20 [==============================] - 154s 6s/step - loss: 2.4863 - sparse_categorical_accuracy: 0.2775 - val_loss: 2.5022 - val_sparse_categorical_accuracy: 0.2100 - _timestamp: 1658775741.0000 - _runtime: 75085.0000
Epoch 645/10000
20/20 [==============================] - 112s 6s/step - loss: 2.4838 - sparse_categorical_accuracy: 0.2050 - val_loss: 2.5477 - val_sparse_categorical_accuracy: 0.2200 - _timestamp: 1658775853.0000 - _runtime: 75197.0000
Epoch 646/10000
20/20 [==============================] - 112s 6s/step - loss: 2.5354 - sparse_categorical_accuracy: 0.1800 - val_loss: 2.5663 - val_sparse_categorical_accuracy: 0.2100 - _timestamp: 1658775965.0000 - _runtime: 75309.0000
Epoch 647/10000
20/20 [==============================] - 113s 6s/step - loss: 2.4900 - sparse_categorical_accuracy: 0.2100 - val_loss: 2.4967 - val_sparse_categorical_accuracy: 0.2500 - _timestamp: 1658776078.0000 - _runtime: 75422.0000
Epoch 648/10000
20/20 [=============================

In [ ]:
run.finish()

In [ ]:
previous_history = history

In [ ]:
model.save_weights("./Saved_Models/Str_Xl_Induced.h5")

In [33]:
model.load_weights("./Saved_Models/Str_Xl_Induced.h5")

In [ ]:
u = model.evaluate(val_dataset)

In [ ]:
u = model.predict(train_dataset)

In [ ]:
u = u.argmax(-1)

In [ ]:
u

In [ ]:
val_dataset[0][1]

In [ ]:
trimmed_samples[0:5]